In [4]:
import lineRenderWorldSetup as LR
import cv2
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt
from glob import glob
from natsort import natsorted
import os
import _3dTo2dSetup as D3
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from scipy.ndimage import zoom

In [5]:
# Load projections
path = r"model_uncertainty\rotated\offset_5" #CHANGE NAME BASED ON OFFSET!
proj_path = natsorted(glob(os.path.join(path, "*.npy")))
box_dim = LR.BOX_DIM
real_box_dim = 301

#load 3D model
path_real = r"solid_400001-segmentation-solid_400001-label.nrrd"
real_box = D3.get3dFigure(real_box_dim, path_real)
real_box = np.where(zoom(np.copy(real_box),1.4) >= 1, 1, 0)

# order projection based on custom selection (see report)
print(proj_path)

['model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_0-5.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_00.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_5-5.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_05.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_50.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_55.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_-5-5.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_-50.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_-55.npy']


In [6]:
#place 0 rotation box as first elm in list (for rotation only)
temp = [proj_path[1]]
for i in range(len(proj_path)):
  if i != 1:
    temp.append(proj_path[i])
proj_path = temp

print(proj_path)

['model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_00.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_0-5.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_5-5.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_05.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_50.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_55.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_-5-5.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_-50.npy', 'model_uncertainty\\rotated\\offset_5\\R_-00_-00_5_-55.npy']


In [7]:
def dice_score(true, pred):
  true_bool = np.asarray(true).astype(np.bool)
  pred_bool = np.asarray(pred).astype(np.bool)
  inter = np.logical_and(true_bool,pred_bool)

  return (2* inter.sum()) / (true_bool.sum()
    + pred_bool.sum())

In [8]:
#add padding for reconstruction comparison
padding = int((box_dim - real_box.shape[0])/2)
print(padding)
real_box = np.pad(real_box, [(padding, padding), (padding, padding),(padding, padding)], mode='constant')

12


In [9]:
#Calculate metrics for intersection of projections
np.set_printoptions(precision=5, suppress = True)
metrics= []
base_proj = np.load(r"model_uncertainty\normal\1_N-87_+27.npy")

for i in range(1,len(proj_path)): #skip first as this contains 0 offset
  #calculate box intersection
  base_box = np.copy(base_proj)
  current_box = np.load(proj_path[i])
  box_total = base_box + current_box
  box_total = np.where(box_total == 2, 1, 0)

  #calculate metrics for each reconstruction
  TN, FP, FN, TP = confusion_matrix(real_box.flatten(), box_total.flatten()).ravel()
  total = TN + FP + FN + TP
  accuracy = (TP + TN)/(FP+FN+TP+TN)
  precision = TP/(TP+FP)
  recall = TP/(TP+FN)
  dice = dice_score(real_box.flatten(), box_total.flatten())
  #f1 = 2*(precision*recall)/(precision+recall)
  scores = np.array([TN/total, FP/total, FN/total, TP/total, dice, precision, recall])
  metrics.append(scores)
  print("Metrics using ", str(i+1) + "nd" , " projection: [tn, fp, fn, tp, dice, precision, recall]", scores)

Metrics using  2nd  projection: [tn, fp, fn, tp, dice, precision, recall] [0.9499  0.03249 0.00044 0.01716 0.51029 0.3456  0.97483]
Metrics using  3nd  projection: [tn, fp, fn, tp, dice, precision, recall] [0.94934 0.03306 0.00067 0.01693 0.50096 0.33869 0.9617 ]
Metrics using  4nd  projection: [tn, fp, fn, tp, dice, precision, recall] [0.94831 0.03409 0.00037 0.01723 0.50003 0.33577 0.97894]
Metrics using  5nd  projection: [tn, fp, fn, tp, dice, precision, recall] [0.94891 0.03348 0.00052 0.01708 0.50111 0.33779 0.97021]
Metrics using  6nd  projection: [tn, fp, fn, tp, dice, precision, recall] [0.9486  0.0338  0.00063 0.01697 0.49646 0.3343  0.96413]
Metrics using  7nd  projection: [tn, fp, fn, tp, dice, precision, recall] [0.95036 0.03203 0.00057 0.01704 0.51105 0.34719 0.96778]
Metrics using  8nd  projection: [tn, fp, fn, tp, dice, precision, recall] [0.94914 0.03326 0.00049 0.01712 0.50363 0.33981 0.97243]
Metrics using  9nd  projection: [tn, fp, fn, tp, dice, precision, recall] [0

In [10]:
# average metric accross the different projections
metrics = np.array(metrics)
avg_metric = np.zeros(len(metrics[0]))
for metric in metrics:
    avg_metric += metric

avg_metric = avg_metric/len(metrics)

print("Average metrics for given offset: [tn, fp, fn, tp, dice, precision, recall]", scores)

Average metrics for given offset: [tn, fp, fn, tp, dice, precision, recall] [0.94832 0.03408 0.00046 0.01714 0.49817 0.33469 0.97386]
